# Denoising Diffusion Probabilistic Models (DDPMs) for Profile Generation (PG)

In [ ]:
import math
import torch
import numpy as np
import pandas as pd  
import matplotlib.pyplot as plt
from IPython.display import HTML
from IPython.display import clear_output
import matplotlib.animation as animation

from dl_models import Model
from data_process import DataSet
from utils import AverageMeter, ProgressMeter
from diffusion_process import DiffusionProcess
from visualizations import scatter, update_plot


In [ ]:

def loss_fn(self, x, idx=None):
    # Calculate the loss function
    output, epsilon, alpha_bar = self.forward(x, idx=idx, get_target=True)
    loss = (output - epsilon).square().mean()
    return loss

In [ ]:
import torch  
import math  
  
# Assuming these are defined in your code  
n_steps = 96  
input_dim = 1
nhead = 16
num_layers = 3
hidden_dim = 64
batch_size=32
  
# Create an instance of the model  
model = Backbone(n_steps, input_dim, hidden_dim=hidden_dim, nhead=nhead, num_layers=num_layers)  
  
# Create a tensor of random numbers to represent your input data  
input_data = torch.randn(batch_size, n_steps, input_dim)  
  
# Create a tensor of random numbers to represent your index  
index = torch.randint(high=n_steps, size=(batch_size,))  
  
# Run the data through the model  
output = model(input_data, index)  
  
# Print the output  
print(output.shape) 



In [ ]:
class Model(nn.Module):
    def __init__(self, device, beta_1, beta_T, T, input_dim):
        '''
        Parameters:
        device: str or torch.device object
            The device to use for the model
        beta_1: float
            The value of beta at time t = 1
        beta_T: float
            The value of beta at time t = T (the final time)
        T: int
            The number of time steps
        input_dim: int
            The input dimension of the model
        '''
        super().__init__()
        self.device = device
        self.T = T

        
        def cosine_schedule(T, s):
            t_vec = np.arange(T)
            f_t = np.cos([(t/T + s)/(1+s) * np.pi/2 for t in t_vec])**2
            alphabar_t = f_t/ f_t[0]
            return alphabar_t
        # Calculate alpha bars
        self.alpha_bars = torch.Tensor(cosine_schedule(T,200)).to(device = device)
        #self.alpha_bars = torch.cumprod(1 - torch.linspace(start = beta_1, end=beta_T, steps=T), dim = 0).to(device = device)
        
        # Create the backbone module
        #file_dir = "first_attempt_model.pth"
        #self.backbone = torch.load(file_dir)
        # Assuming these are defined in your code  
        n_steps = 96  
        nhead = 16
        num_layers = 3
        hidden_dim = 64
        
        self.backbone = Backbone(n_steps=n_steps, input_dim=1, 
                                 hidden_dim=hidden_dim, nhead=nhead, 
                                 num_layers=num_layers, T=T)

        # Move the model to the device
        self.to(device = self.device)


    def loss_fn(self, x, idx=None):
        # Calculate the loss function
        output, epsilon, alpha_bar = self.forward(x, idx=idx, get_target=True)
        loss = (output - epsilon).square().mean()
        return loss

    def forward(self, x, idx=None, get_target=False):
        if not isinstance(idx, int):
            idx = torch.randint(0, self.T, (x.size(0), )).to(device = self.device)
            #print(idx.shape)
            used_alpha_bars = self.alpha_bars[idx[:, None]][:, None]
            #print(used_alpha_bars.shape)
            epsilon = torch.randn_like(x)
            x_tilde = torch.sqrt(used_alpha_bars) * x + torch.sqrt(1 - used_alpha_bars) * epsilon
        else:
            x_tilde = x
            idx = torch.full((x.shape[0], ), idx)
        
        
        #print(x_tilde.shape, idx.shape)
        # Pass x_tilde and idx through the backbone module
        output = self.backbone(x_tilde, idx)

        return (output, epsilon, used_alpha_bars) if get_target else output

In [ ]:
# Set the values for beta_1, beta_T, T, shape, device
beta_1 = 1e-4
beta_T = 0.02
T = 1000
shape = (96, 1)
device = "cpu" #torch.device('cuda')


# Create the Model instance and the Diffusion Process
model = Model(device, beta_1, beta_T, T, 1)

# Run the data through the model  
output = model(input_data, index)
output = model(input_data)

print(output.shape)

In [ ]:
# Define the DiffusionProcess class
class DDPM():

    # Initialize the class
    def __init__(self, beta_1, beta_T, T, diffusion_fn, device, shape):
        '''
        
        Parameters:
        beta_1: float
            The value of beta at time t = 1
        beta_T: float
            The value of beta at time t = T (the final time)    
        T: int
            The number of time steps
        diffusion_fn: function
            The function that defines the diffusion process
        device: str or torch.device object  
            The device to use for the diffusion process
        shape: tuple    
            The shape of the diffusion process
        '''
        self.betas = torch.linspace(start = beta_1, end=beta_T, steps=T)
        self.alphas = 1 - self.betas
        self.T = T
        self.alpha_bars = torch.cumprod(1 - torch.linspace(start = beta_1, 
                                                           end=beta_T,
                                                           steps=T), 
                                        dim = 0).to(device = device)
        self.alpha_prev_bars = torch.cat([torch.Tensor([1]).to(device=device), self.alpha_bars[:-1]])
        self.shape = shape
        self.diffusion_fn = diffusion_fn
        self.device = device
        
    def q_xt(self, x0, t):
        mu = x0 * np.sqrt(self.alpha_bars[t])
        sigma = 1 - self.alpha_bars[t]
        return mu + sigma * np.random.randn(*x0.shape)

    def _one_diffusion_step(self, x):
        '''
        
        Parameters:
        x: torch.Tensor
            The input tensor for the diffusion process 
        '''
        # Perform one diffusion step
        for idx in reversed(range(self.T)):
            noise = torch.zeros_like(x) if idx == 0 else torch.randn_like(x)
            sqrt_tilde_beta = torch.sqrt((1 - self.alpha_prev_bars[idx]) / (1 - self.alpha_bars[idx]) * self.betas[idx])
            
            idx_t = torch.Tensor([int(idx) for _ in range(x.size(0))]).to(device = self.device).long()
            
            idx_t_repeated = idx_t.repeat(0, 96)  
            idx_t = idx_t_repeated[:, :, None]  
            
            #print(x.shape, idx_t.shape)
            predict_epsilon = self.diffusion_fn(x, idx_t)
            mu_theta_xt = torch.sqrt(1 / self.alphas[idx]) * (x - self.betas[idx] / torch.sqrt(1 - self.alpha_bars[idx]) * predict_epsilon)
            x = mu_theta_xt + sqrt_tilde_beta * noise
            yield x

    @torch.no_grad()
    def sampling(self, sampling_number, only_final=False):
        # Perform sampling from the diffusion process
        sample = torch.randn([sampling_number,*self.shape]).to(device = self.device).squeeze()
        sample = sample[:, :, None]
        sampling_list = []

        final = None
        for idx, sample in enumerate(self._one_diffusion_step(sample)):
            final = sample
            if not only_final:
                sampling_list.append(final)

        return final if only_final else torch.stack(sampling_list)

In [ ]:
df = pd.read_csv('../../../../data_exploration/data.csv')  
  
transformed_df = df.copy()  
transformed_df['Id'] = transformed_df['Id'].astype(str)  
transformed_df['Timestamp'] = pd.to_datetime(transformed_df['Timestamp'])  
  
# Append the date to the Id before converting the 'Timestamp' to time  
transformed_df['Id'] = transformed_df['Id'] + ' - ' + transformed_df['Timestamp'].dt.date.astype(str)  
transformed_df['Timestamp'] = transformed_df['Timestamp'].dt.time  
  
# Group by 'Id' and 'Timestamp' and calculate the mean of 'Energy_Consumption'  
transformed_df = transformed_df.groupby(['Id', 'Timestamp'])['Energy_Consumption'].mean().reset_index()  
  
# Pivot the table  
transformed_df = transformed_df.pivot(index='Id', columns='Timestamp', values='Energy_Consumption') 

# Drop the NaN values
transformed_df = transformed_df.dropna()

In [ ]:
from scipy.ndimage.interpolation import rotate
from IPython.display import HTML
from IPython.display import clear_output
from functools import partial

import matplotlib.pyplot as plt
import matplotlib.animation as animation



In [ ]:
class DataSet(torch.utils.data.Dataset):      
    def __init__(self, df):  
        #df = df.diff()  
        df = df.dropna()  # drop NaN values after diff operation  
        self.df = self.scale(df)  
        self.total_len = len(self.df)  
      
    def scale(self, df):    
        # Scale the data    
        df_scaled = pd.DataFrame(df, columns=df.columns, index=df.index)  
        df_scaled.values /=
        return df_scaled    
    
    def inverse_scale(self, df):    
        # Inverse scale the data    
        df_inv_scaled = pd.DataFrame(self.inverse_scaler(df), columns=df.columns, index=df.index)    
        return df_inv_scaled   
    
    def __len__(self):    
        return self.total_len    
    
    def __getitem__(self, idx):      
        data = self.df.iloc[idx].values    
        return torch.from_numpy(data).float().unsqueeze(-1)   


transformed_df = transformed_df[(transformed_df <= 10).all(axis=1)]  

# Create a DataSet instance  
dataset = DataSet(transformed_df)  


dataloader = torch.utils.data.DataLoader(dataset,
                                        batch_size = batch_size, 
                                        drop_last = True)

dataiterator = iter(dataloader)


In [ ]:
transformed_df.shape

In [ ]:
def cosine_schedule(T, s):
    t_vec = np.arange(T)
    f_t = np.cos([(t/T + s)/(1+s) * np.pi/2 for t in t_vec])**2
    alphabar_t = f_t/ f_t[0]
    beta_t = np.array([min(1 - a_t/a_t1, 0.999) for a_t, a_t1 in zip(alphabar_t[1:], alphabar_t[:-1])])
    alpha_t = 1 - beta_t
    return beta_t, alpha_t, alphabar_t

def linear_schedule(beta_min, beta_max, T):
    beta_t = np.linspace(beta_min, beta_max, T)
    alpha_t = 1 - beta_t
    alphabar_t = np.cumprod(alpha_t)
    return beta_t, alpha_t, alphabar_t
     

In [ ]:
beta_min, beta_max = 1e-5, 0.02
T = 100
s = 200

beta_t_lin, alpha_t_lin, alphabar_t_lin = linear_schedule(beta_min, beta_max, T)  
beta_t_cos, alpha_t_cos, alphabar_t_cos = cosine_schedule(T, s) 



plt.figure(figsize=(15,5))  
  
plt.subplot(1,3,1)  
plt.plot(beta_t_cos, label='Cosine')  
plt.plot(beta_t_lin, label='Linear')  
plt.title('Beta')  
plt.legend()  
  
plt.subplot(1,3,2)  
plt.plot(alpha_t_cos, label='Cosine')  
plt.plot(alpha_t_lin, label='Linear')  
plt.title('Alpha')  
plt.legend()  
  
plt.subplot(1,3,3)  
plt.plot(alphabar_t_cos, label='Cosine')  
plt.plot(alphabar_t_lin, label='Linear')  
plt.title('Alpha Bar')  
plt.legend()  
  
plt.tight_layout()  
plt.show()  
import matplotlib.pyplot as plt  
  
# Assuming tensor is your input tensor
tensor = next(iter(dataloader))

In [ ]:
import matplotlib.pyplot as plt  
  
# Assuming tensor is your input tensor
tensor = next(iter(dataloader))

tensor_numpy = tensor.detach().cpu().numpy()

In [ ]:
# Set the values for beta_1, beta_T, T, shape, device
shape = (96, 1)


process = DDPM(beta_1, beta_T, T, model, device, shape=shape)

t = int((T-1)/2)

output_t = process.q_xt(tensor, t)

# Convert tensor to numpy array  
numpy_data = output_t.numpy()  



# Create a figure and a set of subplots  
fig, ax = plt.subplots()  

#scat = plt.scatter(scatter_x, scatter_y, s=1)

# This will create a line for each row in your data. If that's too many lines,  
# you might need to select a subset of rows to plot.  
for i in range(numpy_data.shape[0]):  
    ax.plot(numpy_data[i, :], alpha=0.1)  
  
plt.show()  

In [ ]:
def q_xt(x0, t, alpha_bars):
        mu = x0 * np.sqrt(alpha_bars[t])
        sigma = 1 - alpha_bars[t]
        return mu + sigma * np.random.randn(*x0.shape)

## Linear Schedule

In [ ]:
from celluloid import Camera    

fig, ax = plt.subplots()  
camera = Camera(fig)  
for t in range(0, T, 10):
    for sample in range(tensor_numpy.shape[0]):
        time_sample_t = q_xt(tensor_numpy[sample], t, alphabar_t_lin)
        ax.plot(time_sample_t, alpha=0.1)  
    
    # Add text showing frequency  
    text = ax.text(0.8, .8, f'Time: {t}', transform=ax.transAxes) 
    camera.snap()  
    
    
animation = camera.animate()  
  
# Save as GIF  
animation.save('evolution_lin.gif', writer='pillow')  
  
plt.show()  


In [ ]:
from IPython.display import Image  
Image(filename="evolution_lin.gif")  


## Cosine schedule

In [ ]:
from celluloid import Camera    

fig, ax = plt.subplots()  
camera = Camera(fig)  
for t in range(0, T, 10):
    for sample in range(tensor_numpy.shape[0]):
        time_sample_t = q_xt(tensor_numpy[sample], t, alphabar_t_cos)
        ax.plot(time_sample_t, alpha=0.1)  
    
    # Add text showing frequency  
    text = ax.text(0.8, 0.8, f'Time: {t}', transform=ax.transAxes) 
    camera.snap()  
    
    
animation = camera.animate()  
  
# Save as GIF  
animation.save('evolution_cos.gif', writer='pillow')  
  
plt.show()  


In [ ]:
from IPython.display import Image  
Image(filename="evolution_cos.gif")  


## Random signal

In [ ]:
from celluloid import Camera    

fig, ax = plt.subplots()  
camera = Camera(fig)  
for t in range(0, T, 1000):
    for sample in range(tensor_numpy.shape[0]):
        time_sample_t = np.random.randn(96)
        ax.plot(time_sample_t, alpha=0.1)  
    
    # Add text showing frequency  
    text = ax.text(0.8, .8, f'Time: {t}', transform=ax.transAxes) 
    camera.snap()  
    
    
animation = camera.animate()  
  
# Save as GIF  
animation.save('evolution_random.gif', writer='pillow')  
  
plt.show()  


In [ ]:
from IPython.display import Image  
Image(filename="evolution_random.gif")  


# Train

In [ ]:


# Set the values for beta_1, beta_T, T, shape, device
beta_1 = 1e-5
beta_T = 0.02
T = 100
shape = (96, 1)
device = torch.device('cuda')


# Create the Model instance and the Diffusion Process
#model = Model(device, beta_1, beta_T, T, 1)
#process = DiffusionProcess(beta_1, beta_T, T, model, device, shape=shape)
#process = DDPM(beta_1, beta_T, T, model, device, shape=shape)
optim = torch.optim.Adam(model.parameters(), lr = 0.0001)


In [ ]:
# Set parameters for training
total_iteration = 10000000
current_iteration = 131000
t_range = 10000
display_iteration = [i for i in range(1, total_iteration, t_range)]
sampling_number = 50
only_final = True

# Set parameters for visualization of the training process
losses = AverageMeter('Loss', ':.4f')
progress = ProgressMeter(total_iteration, [losses], prefix='Iteration ')


# Start the training 
while current_iteration != total_iteration:
    try:
        data = next(dataiterator)
    except:
        dataiterator = iter(dataloader)
        data = next(dataiterator)
    
    data = data.to(device = device)
    loss = model.loss_fn(data)

    optim.zero_grad()
    loss.backward()
    
     # `clip_value` is the upper limit for the gradient.  
    # This line does the actual clipping of gradients.  
    #torch.nn.utils.clip_grad_norm_(model.parameters(), 1)  
  
    # Proceed with the optimization step 
    optim.step()

     # Check if the loss is NaN or Inf  
    if torch.isnan(loss).item() or torch.isinf(loss).item():  
        print("The loss is NaN or Inf")  
        print(data)
        print(model(data))
        break  
        
    losses.update(loss.item())
    progress.display(current_iteration)
    current_iteration += 1

    if current_iteration in display_iteration:
        process = DDPM(beta_1, beta_T, T, model, device, shape)
        sample = process.sampling(sampling_number, only_final)
        sample = sample.detach().cpu().numpy()
        
        # Create a figure and a set of subplots  
        fig, ax = plt.subplots()  

        # This will create a line for each row in your data. If that's too many lines,  
        # you might need to select a subset of rows to plot.  
        for i in range(sample.shape[0]):  
            ax.plot(sample[i, :], alpha=0.1)  

        plt.show()  
        losses.reset()
        torch.save(model.backbone, 'good_noise_model1.pth')
        
torch.save(model.backbone, 'good_noise_model1.pth') 

In [ ]:
process = DDPM(beta_1, beta_T, T, model, device, shape)
sample = process.sampling(sampling_number, only_final=False)
sample = sample.detach().cpu().numpy()


fig, ax = plt.subplots()  
camera = Camera(fig)  
for t in range(0, T, 10):
    for i in range(tensor_numpy.shape[0]):
        ax.plot(sample[t, i, :], alpha=0.1)
    
    # Add text showing frequency  
    text = ax.text(0.8, 0.8, f'Time: {t}', transform=ax.transAxes) 
    camera.snap()  
    
    
animation = camera.animate()  
  
# Save as GIF  
animation.save('evolution_ddpm.gif', writer='pillow')  
  
plt.show()  


In [ ]:
from IPython.display import Image  
Image(filename="evolution_ddpm.gif")  


In [ ]:
# Define the DiffusionProcess class
class DDPM():

    # Initialize the class
    def __init__(self, beta_1, beta_T, T, diffusion_fn, device, shape):
        '''
        
        Parameters:
        beta_1: float
            The value of beta at time t = 1
        beta_T: float
            The value of beta at time t = T (the final time)    
        T: int
            The number of time steps
        diffusion_fn: function
            The function that defines the diffusion process
        device: str or torch.device object  
            The device to use for the diffusion process
        shape: tuple    
            The shape of the diffusion process
        '''
        def cosine_schedule(T, s):
            t_vec = np.arange(T)
            f_t = np.cos([(t/T + s)/(1+s) * np.pi/2 for t in t_vec])**2
            alphabar_t = f_t/ f_t[0]
            beta_t = np.array([min(1 - a_t/a_t1, 0.999)] + [min(1 - a_t/a_t1, 0.999) for a_t, a_t1 in zip(alphabar_t[1:], alphabar_t[:-1])])
            alpha_t = 1 - beta_t
            return beta_t, alpha_t, alphabar_t
        
        beta, alpha, alphabar = cosine_schedule(T, 200)
        print(beta.shape, alpha.shape, alphabar.shape)
        # Calculate alpha bars
        self.alpha_bars = torch.Tensor(alphabar).to(device = device)
        self.betas = torch.Tensor(beta).to(device = device)
        self.alphas = torch.Tensor(alpha).to(device = device)
        #self.alpha_bars = torch.cumprod(1 - torch.linspace(start = beta_1, end=beta_T, steps=T), dim = 0).to(devi
        #self.betas = torch.linspace(start = beta_1, end=beta_T, steps=T)
        #self.alphas = 1 - self.betas
        self.T = T
        #self.alpha_bars = torch.cumprod(1 - torch.linspace(start = beta_1, 
         #                                                  end=beta_T,
          #                                                 steps=T), 
           #                             dim = 0).to(device = device)
        
        self.alpha_prev_bars = torch.cat([torch.Tensor([1]).to(device=device), self.alpha_bars[:-1]])
        self.shape = shape
        self.diffusion_fn = diffusion_fn
        self.device = device
        
    def q_xt(self, x0, t):
        mu = x0 * np.sqrt(self.alpha_bars[t])
        sigma = 1 - self.alpha_bars[t]
        return mu + sigma * np.random.randn(*x0.shape)

    def _one_diffusion_step(self, x):
        '''
        
        Parameters:
        x: torch.Tensor
            The input tensor for the diffusion process 
        '''
        # Perform one diffusion step
        for idx in reversed(range(self.T)):
            noise = torch.zeros_like(x) if idx == 0 else torch.randn_like(x)
            sqrt_tilde_beta = torch.sqrt((1 - self.alpha_prev_bars[idx]) / (1 - self.alpha_bars[idx]) * self.betas[idx])
            
            idx_t = torch.Tensor([int(idx) for _ in range(x.size(0))]).to(device = self.device).long()
            
            idx_t_repeated = idx_t.repeat(1, 96)  
            idx_t = idx_t_repeated[:, :, None]  
            
            #print(x.shape, idx_t.shape)
            predict_epsilon = self.diffusion_fn(x, idx_t)
            mu_theta_xt = torch.sqrt(1 / self.alphas[idx]) * (x - self.betas[idx] / torch.sqrt(1 - self.alpha_bars[idx]) * predict_epsilon)
            x = mu_theta_xt 
            #+ sqrt_tilde_beta * noise
            yield x

    @torch.no_grad()
    def sampling(self, sampling_number, only_final=False):
        # Perform sampling from the diffusion process
        sample = torch.randn([sampling_number,*self.shape]).to(device = self.device).squeeze()
        sample = sample[:, :, None]
        sampling_list = []

        final = None
        for idx, sample in enumerate(self._one_diffusion_step(sample)):
            final = sample
            if not only_final:
                sampling_list.append(final)

        return final if only_final else torch.stack(sampling_list)

In [ ]:
numpy_data.shape

In [ ]:
sample.shape

In [ ]:
sampling_number = 100
only_final = True

process = DDPM(beta_1, beta_T, T, model, device, shape=shape)
sample = process.sampling(sampling_number, only_final)
#scatter(sample[9::30], only_final)

sample = sample.detach().cpu().numpy()

# Create a figure and a set of subplots  
fig, ax = plt.subplots()  
  
# This will create a line for each row in your data. If that's too many lines,  
# you might need to select a subset of rows to plot.  
for i in range(sample.shape[1]):  
    ax.plot(sample[ i, :], alpha=0.1)  
  
plt.show()  

In [ ]:
sampling_number = 100
only_final = True


# Set the values for beta_1, beta_T, T, shape, device
beta_1 = 1e-4
beta_T = 0.02
T = 
shape = (96, 1)
device = torch.device('cuda')



process = DDPM(beta_1, beta_T, T, model, device, shape=shape)
sample = process.sampling(sampling_number, only_final)
#scatter(sample[9::30], only_final)

sample = sample.detach().cpu().numpy()

# Create a figure and a set of subplots  
fig, ax = plt.subplots()  
  
# This will create a line for each row in your data. If that's too many lines,  
# you might need to select a subset of rows to plot.  
for i in range(sample.shape[1]):  
    ax.plot(sample[ i, :], alpha=0.1)  
  
plt.show()  

In [ ]:
numframes = len(sample)
scatter_point = sample[0].detach().cpu().numpy()
scatter_x, scatter_y = scatter_point[:,0], scatter_point[:,1]


scatter_range = [-10, 10]
fig = plt.figure(figsize=(6, 6))
plt.xlim(scatter_range)
plt.ylim(scatter_range)
scat = plt.scatter(scatter_x, scatter_y, s=1)
plt.show()
clear_output()

ani = animation.FuncAnimation(fig, update_plot, frames=range(numframes), fargs=(sample, scat), interval=50)
writergif = animation.PillowWriter(fps=40)

ani.save('ddpm_toy1.gif', writer=writergif)
HTML(ani.to_jshtml())


In [ ]:
sample

In [ ]:
def count_parameters(model):  
    return sum(p.numel() for p in model.parameters() if p.requires_grad)  
  
model = Backbone(n_steps=1000, input_dim=1, nhead=16, num_layers=4)  
model = model.to(device)  
  
print(f"The model has {count_parameters(model):,} trainable parameters")  


In [ ]:
# Instantiate the model    
model = Backbone(n_steps=1000, input_dim=1, nhead=16, num_layers=4)    
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')    
    
# Create dummy data with the correct shape    
batch_size = 256
x = torch.randn(batch_size, 96, 1).to('cuda' if torch.cuda.is_available() else 'cpu')  # Batch size is now 1    
idx = torch.randint(0, 1000, (batch_size, 1)).to('cuda' if torch.cuda.is_available() else 'cpu')  
    
# Run the data through your model    
output = model(x, idx)    
  
print("Output shape:", output.shape)  
